In [ ]:
import pandas as pd
import numpy as np

import folium 
from matplotlib.pyplot import cm
import matplotlib

import gpxpy
import gpxpy.gpx

In [ ]:
file_name = 'data/gpx/Course_pied_le_soir.gpx'

In [ ]:
def parse_gpx(file_name):
    gpx = gpxpy.parse(open(file_name)) 

    track = gpx.tracks[0]
    segment = track.segments[0]
    data = []
    segment_length = segment.length_3d()
    for point_idx, point in enumerate(segment.points):
        data.append([point.longitude, point.latitude,point.elevation,
        point.time, segment.get_speed(point_idx)*3.6])

    columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
    return pd.DataFrame(data, columns=columns)

gpx_df = parse_gpx(file_name)


In [ ]:
gpx_df['RollSpeed'] = gpx_df['Speed'].rolling(10, min_periods= 0).mean()

In [ ]:
mymap = folium.Map(location=[gpx_df.Latitude.mean(), gpx_df.Longitude.mean()], zoom_start=12, tiles=None)
folium.TileLayer('openstreetmap', name='OpenStreet Map').add_to(mymap)
folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}', attr='Tiles &copy; Esri &mdash; National Geographic, Esri, DeLorme, NAVTEQ, UNEP-WCMC, USGS, NASA, ESA, METI, NRCAN, GEBCO, NOAA, iPC', name='Nat Geo Map').add_to(mymap)
folium.TileLayer('http://tile.stamen.com/terrain/{z}/{x}/{y}.jpg', attr='terrain-bcg', name='Terrain Map').add_to(mymap)

In [ ]:
def make_get_speed_color(max_speed=20, min_speed=5, cmap=cm.get_cmap('inferno')):
    def get_speed_color(speed):
        intensity = min(1, max(0, (speed-min_speed)/max_speed))
        rgba = cmap(int(intensity*cmap.N))
        return matplotlib.colors.rgb2hex(rgba)
    return get_speed_color

In [ ]:
speed_col_conv = make_get_speed_color()
for i in range(gpx_df.shape[0]-1):
    a = gpx_df.iloc[i][['Latitude', 'Longitude']].tolist()
    b = gpx_df.iloc[i+1][['Latitude', 'Longitude']].tolist()
    speed = gpx_df.iloc[i]['RollSpeed']
    folium.PolyLine([a, b], color=speed_col_conv(speed), weight=7, opacity=1).add_to(mymap)


In [ ]:
mymap

In [ ]:
cm.get_cmap('inferno')

---

End Of Notebook

---